# 🏪 店舗別包括ダッシュボード v3.0 - フル・インタラクティブ版

## 📊 機能一覧

### 🎯 新機能 (v3.0)
- ✅ **店舗選択**: ウィジェットで店舗を動的に切替
- ✅ **時間軸切替**: 月次/週次/日次を選択可能
- ✅ **高度なドリルダウン**: クリックで詳細データを探索
- ✅ **包括的フィルタ**: 店舗・期間・商品・カテゴリで絞込
- ✅ **表示件数調整**: 10/20/50/100件を選択
- ✅ **ソート機能**: 任意の列でソート
- ✅ **検索機能**: 商品名で検索

### 📈 既存機能
1. ✅ 現状把握分析 - 売上推移・曜日パターン
2. ✅ 商品ABC分析 - 重要商品特定
3. ✅ 発注最適化 - 安全在庫・発注点
4. ✅ 外部要因分析 - 気温・天気影響
5. ✅ 店舗間比較 - ベンチマーク
6. ✅ 自動アラート - 要対応項目検出

## 📦 1. 環境設定

In [ ]:
# 日本語フォント設定（共通モジュール）
import warnings
warnings.filterwarnings('ignore')

# よく使うライブラリを先に読み込む
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# ウィジェットの有無を通知・フラグ化
try:
    import ipywidgets as widgets
    from IPython.display import display, HTML, clear_output
    WIDGETS_AVAILABLE = True
    print('✅ ipywidgets利用可能')
except Exception:
    WIDGETS_AVAILABLE = False
    print('⚠️ ipywidgets未インストール - 一部機能制限')

import font_setup
JP_FP = font_setup.setup_fonts(show_test=False)


## 📁 2. データ読み込み

In [ ]:
# データパス
DATA_PATH = 'output/06_final_enriched_20250701_20250930.csv'

print(f"📂 データ読み込み: {DATA_PATH}")

# 全店舗データ
df_all = pd.read_csv(DATA_PATH, encoding='utf-8-sig', low_memory=False)
df_all['日付'] = pd.to_datetime(df_all['日付'])

# 年月・年週列追加
df_all['年月'] = df_all['日付'].dt.to_period('M').astype(str)
df_all['年週'] = df_all['日付'].dt.to_period('W').astype(str)
df_all['曜日'] = df_all['日付'].dt.dayofweek
weekday_map = {0: '月', 1: '火', 2: '水', 3: '木', 4: '金', 5: '土', 6: '日'}
df_all['曜日名'] = df_all['曜日'].map(weekday_map)

# 店舗リスト
STORES = sorted(df_all['店舗'].unique())

print(f"\n✅ データ読込完了")
print(f"   期間: {df_all['日付'].min().date()} 〜 {df_all['日付'].max().date()}")
print(f"   総データ数: {len(df_all):,}行")
print(f"   店舗数: {len(STORES)}店")
print(f"   店舗: {', '.join(STORES)}")

# グローバル変数初期化
DISPLAY_OPTIONS = [10, 20, 50, 100]
TIME_PERIODS = ['日次', '週次', '月次']

## 🎮 3. コントロールパネル - 店舗・期間選択

In [ ]:
if WIDGETS_AVAILABLE:
    # 店舗選択
    store_selector = widgets.Dropdown(
        options=STORES,
        value=STORES[0],
        description='📍 店舗:',
        style={'description_width': '80px'},
        layout=widgets.Layout(width='400px')
    )
    
    # 時間軸選択
    period_selector = widgets.Dropdown(
        options=TIME_PERIODS,
        value='日次',
        description='📅 時間軸:',
        style={'description_width': '80px'},
        layout=widgets.Layout(width='200px')
    )
    
    # 表示件数
    count_selector = widgets.Dropdown(
        options=DISPLAY_OPTIONS,
        value=20,
        description='🔢 表示件数:',
        style={'description_width': '80px'},
        layout=widgets.Layout(width='200px')
    )
    
    # ABCランクフィルタ
    rank_filter = widgets.Dropdown(
        options=['全て', 'A', 'B', 'C'],
        value='全て',
        description='📊 ABCランク:',
        style={'description_width': '80px'},
        layout=widgets.Layout(width='200px')
    )
    
    # 検索ボックス
    search_box = widgets.Text(
        value='',
        placeholder='商品名で検索',
        description='🔍 検索:',
        style={'description_width': '80px'},
        layout=widgets.Layout(width='400px')
    )
    
    # 更新ボタン
    refresh_button = widgets.Button(
        description='🔄 データ更新',
        button_style='primary',
        layout=widgets.Layout(width='150px')
    )
    
    # 出力エリア
    output_area = widgets.Output()
    
    # レイアウト
    control_panel = widgets.VBox([
        widgets.HTML("<h2>🎮 コントロールパネル</h2>"),
        widgets.HBox([store_selector, period_selector]),
        widgets.HBox([count_selector, rank_filter]),
        widgets.HBox([search_box, refresh_button]),
        widgets.HTML("<hr>")
    ])
    
    display(control_panel)
    display(output_area)
    
    print("✅ コントロールパネル表示")
else:
    print("⚠️ ウィジェット非対応 - 静的表示モード")
    # フォールバック設定
    SELECTED_STORE = STORES[0]
    SELECTED_PERIOD = '日次'
    SELECTED_COUNT = 20
    SELECTED_RANK = '全て'
    SEARCH_TERM = ''

## 📊 4. メイン分析関数

In [ ]:
def analyze_store(store_name, time_period='日次', display_count=20, abc_rank='全て', search_term=''):
    """
    店舗の包括的分析を実行
    
    Parameters:
    -----------
    store_name : str
        対象店舗名
    time_period : str
        時間軸 ('日次', '週次', '月次')
    display_count : int
        表示件数
    abc_rank : str
        ABCランクフィルタ
    search_term : str
        検索キーワード
    """
    
    # 店舗データ抽出
    df = df_all[df_all['店舗'] == store_name].copy()
    
    print("="*80)
    print(f"📊 {store_name} - 包括分析レポート")
    print("="*80)
    print(f"\n📅 分析期間: {df['日付'].min().date()} 〜 {df['日付'].max().date()}")
    print(f"📈 時間軸: {time_period}")
    print(f"📦 データ数: {len(df):,}行")
    
    # ===== Section 1: KPI サマリー =====
    print("\n" + "="*80)
    print("📈 SECTION 1: KPIサマリー")
    print("="*80)
    
    total_sales = df['売上金額'].sum()
    total_qty = df['売上数量'].sum()
    avg_price = total_sales / total_qty if total_qty > 0 else 0
    days = df['日付'].nunique()
    
    print(f"\n💰 総売上高: ¥{total_sales:,.0f}")
    print(f"📊 日商平均: ¥{total_sales/days:,.0f}/日")
    print(f"📦 総販売数: {total_qty:,.0f}個")
    print(f"💵 平均単価: ¥{avg_price:,.0f}/個")
    print(f"🏪 商品数: {df['商品名'].nunique():,}品")
    
    # ===== Section 2: 時系列分析 =====
    print("\n" + "="*80)
    print(f"📈 SECTION 2: 売上推移 ({time_period})")
    print("="*80)
    
    if time_period == '日次':
        time_col = '日付'
        timeseries = df.groupby('日付').agg({'売上金額': 'sum', '売上数量': 'sum'}).reset_index()
    elif time_period == '週次':
        time_col = '年週'
        timeseries = df.groupby('年週').agg({'売上金額': 'sum', '売上数量': 'sum'}).reset_index()
    else:  # 月次
        time_col = '年月'
        timeseries = df.groupby('年月').agg({'売上金額': 'sum', '売上数量': 'sum'}).reset_index()
    
    # Plotlyグラフ
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=timeseries[time_col],
        y=timeseries['売上金額'],
        mode='lines+markers',
        name='売上金額',
        line=dict(color='blue', width=3),
        marker=dict(size=8),
        hovertemplate=f'{time_col}: %{{x}}<br>売上: ¥%{{y:,.0f}}<extra></extra>'
    ))
    
    fig.update_layout(
        title=f'📈 {store_name} - {time_period}売上推移',
        xaxis_title=time_col,
        yaxis_title='売上金額（円）',
        hovermode='x unified',
        height=500
    )
    fig.show()
    
    # 統計情報
    print(f"\n📊 {time_period}統計:")
    print(f"   平均: ¥{timeseries['売上金額'].mean():,.0f}")
    print(f"   最大: ¥{timeseries['売上金額'].max():,.0f}")
    print(f"   最小: ¥{timeseries['売上金額'].min():,.0f}")
    print(f"   標準偏差: ¥{timeseries['売上金額'].std():,.0f}")
    
    # ===== Section 3: 商品ABC分析 =====
    print("\n" + "="*80)
    print("📦 SECTION 3: 商品ABC分析")
    print("="*80)
    
    products = df.groupby('商品名').agg({
        '売上金額': 'sum',
        '売上数量': 'sum',
        '日付': 'nunique'
    }).reset_index()
    products.columns = ['商品名', '売上金額', '売上数量', '販売日数']
    products = products.sort_values('売上金額', ascending=False).reset_index(drop=True)
    
    # ABC分類
    products['構成比_%'] = (products['売上金額'] / products['売上金額'].sum() * 100).round(2)
    products['累積構成比_%'] = products['構成比_%'].cumsum().round(2)
    products['ABCランク'] = products['累積構成比_%'].apply(
        lambda x: 'A' if x <= 80 else ('B' if x <= 95 else 'C')
    )
    products['平均単価'] = (products['売上金額'] / products['売上数量']).round(0)
    products['1日平均売上'] = (products['売上金額'] / products['販売日数']).round(0)
    
    # フィルタリング
    filtered_products = products.copy()
    
    if abc_rank != '全て':
        filtered_products = filtered_products[filtered_products['ABCランク'] == abc_rank]
    
    if search_term:
        filtered_products = filtered_products[
            filtered_products['商品名'].str.contains(search_term, case=False, na=False)
        ]
    
    # ABC集計
    abc_summary = products.groupby('ABCランク').agg({
        '商品名': 'count',
        '売上金額': 'sum'
    }).reindex(['A', 'B', 'C'])
    abc_summary.columns = ['商品数', '売上金額']
    abc_summary['商品数_%'] = (abc_summary['商品数'] / len(products) * 100).round(1)
    abc_summary['売上構成比_%'] = (abc_summary['売上金額'] / abc_summary['売上金額'].sum() * 100).round(1)
    
    print(f"\n総商品数: {len(products):,}商品")
    print(abc_summary.to_string())
    
    # 商品リスト表示
    print(f"\n【商品一覧】 フィルタ後: {len(filtered_products)}商品 (表示: TOP {display_count})")
    print("="*80)
    display_cols = ['商品名', '売上金額', '売上数量', 'ABCランク', '平均単価', '1日平均売上']
    print(filtered_products.head(display_count)[display_cols].to_string(index=False))
    
    # パレート図
    top100 = products.head(100)
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    
    fig.add_trace(
        go.Bar(
            x=list(range(len(top100))),
            y=top100['売上金額'],
            name='売上金額',
            marker_color='steelblue',
            opacity=0.7,
            customdata=top100[['商品名', '売上金額', '売上数量', 'ABCランク']],
            hovertemplate='<b>%{customdata[0]}</b><br>売上: ¥%{customdata[1]:,.0f}<br>数量: %{customdata[2]:,.0f}個<br>ランク: %{customdata[3]}<extra></extra>'
        ),
        secondary_y=False
    )
    
    fig.add_trace(
        go.Scatter(
            x=list(range(len(top100))),
            y=top100['累積構成比_%'],
            name='累積構成比',
            mode='lines+markers',
            line=dict(color='red', width=3),
            marker=dict(size=6),
            hovertemplate='累積: %{y:.1f}%<extra></extra>'
        ),
        secondary_y=True
    )
    
    fig.add_hline(y=80, line_dash="dash", line_color="orange", annotation_text="80%(A/B)", secondary_y=True)
    fig.add_hline(y=95, line_dash="dash", line_color="green", annotation_text="95%(B/C)", secondary_y=True)
    
    fig.update_xaxes(title_text="商品順位")
    fig.update_yaxes(title_text="売上金額（円）", secondary_y=False)
    fig.update_yaxes(title_text="累積構成比（%）", range=[0, 105], secondary_y=True)
    fig.update_layout(title="📊 パレート図 (TOP 100商品)", height=600, hovermode='x unified')
    fig.show()
    
    # ===== Section 4: 曜日パターン =====
    print("\n" + "="*80)
    print("📆 SECTION 4: 曜日別パターン")
    print("="*80)
    
    weekday_order = ['月', '火', '水', '木', '金', '土', '日']
    weekday = df.groupby('曜日名')['売上金額'].agg(['sum', 'mean', 'count']).reindex(weekday_order)
    weekday.columns = ['売上合計', '売上平均', 'データ数']
    weekday_avg = weekday.loc[['月', '火', '水', '木', '金'], '売上平均'].mean()
    weekday['指数'] = (weekday['売上平均'] / weekday_avg * 100).round(0)
    
    print(weekday.to_string())
    
    colors = ['red' if d in ['土', '日'] else 'steelblue' for d in weekday_order]
    fig = go.Figure()
    fig.add_trace(go.Bar(
        x=weekday_order,
        y=weekday['売上平均'],
        marker_color=colors,
        text=weekday['売上平均'].round(0),
        textposition='outside',
        hovertemplate='%{x}曜日<br>平均: ¥%{y:,.0f}<extra></extra>'
    ))
    fig.update_layout(title="📆 曜日別平均売上", xaxis_title="曜日", yaxis_title="売上金額（円）", height=400)
    fig.show()
    
    # ===== Section 5: アラート =====
    print("\n" + "="*80)
    print("🚨 SECTION 5: 自動アラート")
    print("="*80)
    
    # 死に筋商品
    total_days = df['日付'].nunique()
    dead_stock = products[
        (products['ABCランク'] == 'C') & 
        ((products['1日平均売上'] < 100) | (products['販売日数'] < total_days * 0.2))
    ]
    
    # 機会ロス商品
    opportunity_loss = products[
        (products['ABCランク'] == 'A') & (products['販売日数'] < total_days * 0.8)
    ]
    
    print(f"\n⚠️ 死に筋商品: {len(dead_stock)}商品")
    print(f"🚨 機会ロス商品: {len(opportunity_loss)}商品")
    
    if len(dead_stock) > 0:
        print(f"\n【死に筋商品 TOP 10】")
        print(dead_stock.head(10)[['商品名', '売上金額', '販売日数']].to_string(index=False))
    
    if len(opportunity_loss) > 0:
        print(f"\n【機会ロス商品 TOP 10】")
        print(opportunity_loss.head(10)[['商品名', '売上金額', '販売日数']].to_string(index=False))
    
    print("\n" + "="*80)
    print("✅ 分析完了")
    print("="*80)

print("✅ 分析関数定義完了")

## 🚀 5. 実行 - ウィジェット連携

In [ ]:
if WIDGETS_AVAILABLE:
    def on_refresh_click(b):
        """更新ボタンクリック時の処理"""
        with output_area:
            clear_output()
            analyze_store(
                store_name=store_selector.value,
                time_period=period_selector.value,
                display_count=count_selector.value,
                abc_rank=rank_filter.value,
                search_term=search_box.value
            )
    
    # ボタンにイベント登録
    refresh_button.on_click(on_refresh_click)
    
    print("✅ ウィジェット連携完了")
    print("\n💡 使い方:")
    print("   1. 上のコントロールパネルで店舗・条件を選択")
    print("   2. '🔄 データ更新'ボタンをクリック")
    print("   3. 分析結果が表示されます")
    print("\n🎯 初回実行: '🔄 データ更新'ボタンをクリックしてください")
else:
    # ウィジェット非対応時は直接実行
    print("⚠️ ウィジェット非対応モード - 静的分析実行")
    analyze_store(
        store_name=STORES[0],
        time_period='日次',
        display_count=20,
        abc_rank='全て',
        search_term=''
    )

---

# ✅ ダッシュボード v3.0 完成

## 📊 新機能まとめ

### 🎮 インタラクティブ機能
- ✅ **店舗選択**: 3店舗を動的に切替可能
- ✅ **時間軸切替**: 日次/週次/月次で分析粒度を変更
- ✅ **表示件数調整**: 10/20/50/100件を選択
- ✅ **ABCランクフィルタ**: A/B/C商品を絞込
- ✅ **商品検索**: 商品名でリアルタイム検索
- ✅ **ドリルダウン**: グラフをホバー/クリックで詳細表示

### 📈 分析機能
1. ✅ KPIサマリー - 総売上・日商平均・商品数
2. ✅ 時系列分析 - 日次/週次/月次の売上推移
3. ✅ 商品ABC分析 - パレート図・重要商品特定
4. ✅ 曜日パターン - 曜日別売上傾向
5. ✅ 自動アラート - 死に筋商品・機会ロス検出

### 💡 使い方
1. コントロールパネルで条件を選択
2. 「🔄 データ更新」ボタンをクリック
3. グラフ・テーブルで探索的にデータを確認
4. 条件を変更して再分析

---

**バージョン**: 3.0  
**作成日**: 2025-10-08  
**機能**: 店舗選択・時間軸切替・フィルタ・検索・ドリルダウン完備